# Import Libraries

In [39]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import ProjectedGradientDescent
from art.defences.preprocessor import SpatialSmoothing

# Load Data

In [2]:
image_path = "racer.jpg"
img = image.load_img(image_path, target_size=(224, 224))

In [3]:
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)  
img = np.clip(img, 0, 1)

# Model

In [4]:
model = ResNet50V2(weights="imagenet")

In [5]:
model.compile(
    optimizer=Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Attack

In [6]:
mean = [103.939, 116.779, 123.68]

In [46]:
classifier = TensorFlowV2Classifier(
    model=model,
    loss_object=CategoricalCrossentropy(),
    train_step=None,
    input_shape=(224, 224, 3),
    clip_values=(0, 255),
    nb_classes=1000,
    preprocessing=(
        mean, np.array([1.0, 1.0, 1.0])
    )
)

In [48]:
pgd = ProjectedGradientDescent(
    classifier,
    targeted=True,
    max_iter=10,
    eps_step=1,
    eps=5
)

In [49]:
target_class = 10

In [50]:
y_one_hot = np.zeros(1000)
y_one_hot[target_class] = 1.0

In [51]:
y_target = np.expand_dims(y_one_hot, axis=0)

In [52]:
adv_img = pgd.generate(x=img, y=y_target)

PGD - Batches: 0it [00:00, ?it/s]

# Spatial Smoothing

In [61]:
def predict_image(img):
    pred = model.predict(img, verbose=0)
    decoded_preds = decode_predictions(pred, top=5)[0]
    return {"class_name": decoded_preds[0][1], "confidence_score": round(decoded_preds[0][2], 2)}

In [62]:
print("Raw Image Prediction:", predict_image(img))
print("Adversarial Image Prediction:", predict_image(adv_img))

Raw Image Prediction: {'class_name': 'racer', 'confidence_score': 0.73}
Adversarial Image Prediction: {'class_name': 'strainer', 'confidence_score': 0.86}


In [55]:
ss = SpatialSmoothing(window_size=3)

In [56]:
ss_raw, _ = ss(img)

In [57]:
ss_adv, _ = ss(adv_img)

In [58]:
print("Raw Image Prediction:", predict_image(ss_raw))
print("Adversarial Image Prediction:", predict_image(ss_adv))

Raw Image Prediction: {'class_name': 'racer', 'confidence_score': 0.85}
Adversarial Image Prediction: {'class_name': 'strainer', 'confidence_score': 0.99}
